# Assignment 5
In Assignment 5, we consider a merge analysis.
- Description of Assignment 5: [link](https://kohei-kawaguchi.github.io/EmpiricalIO/assignment5.html#estimate-the-parameters-3)

In [2]:
# import packages
import pandas as pd
import numpy as np
import statsmodels.api as sm
from numba import jit, njit, prange
import matplotlib.pyplot as plt
from scipy.optimize import minimize

In [3]:
import numba
import warnings

warnings.simplefilter('ignore', category=numba.errors.NumbaDeprecationWarning)
warnings.simplefilter('ignore', category=numba.errors.NumbaPendingDeprecationWarning)

# 1 Simulate data

## 1.1 Set the seed, constants, and parameters of interest.

In [4]:
# set the seed
np.random.seed(1019)
# number of products
J = 10
# dimension of product characteristics including the intercept (=1)
K = 3
# number of markets
T = 20
# number of consumers per market
N = 100
# number of Monte Carlo
L = 100

- $[\beta_{01}, \cdots, \beta_{0K}]$ is referred to as beta.
- $[\sigma_{1}, \cdots, \sigma_{K}]$ is referred to as sigma.

In [5]:
# set parameters of interest
beta = np.random.standard_normal(size = (K,1))
beta[0] = 4

print('beta: ', beta)

sigma = abs(np.random.standard_normal(size = (K,1))) 
print('sigma: ', sigma)

mu = 0.5
omega = 1

beta:  [[ 4.        ]
 [-0.34712881]
 [-0.64393574]]
sigma:  [[0.62307216]
 [1.0996384 ]
 [1.01118079]]


In [6]:
# set auxiliary parameters
price_xi = 1
sd_x = 2
sd_xi = 0.5
sd_c = 0.05
sd_p = 0.05

## 1.2 X
X is the data frame such that a row contains the characteristics vector $x_j$ of a product and columns are product index and observed product characteristics. The dimension of the characteristics $K$ is specified above. Add the row of the outside option whose index is 0 and all the characteristics are zero.

- The product-market characteristics: $$x_{j1}=1, x_{jk}\sim N(0, \sigma_x), k=2,\cdots,K,$$ where $\sigma_x$ is referred to as sd_x in the code.

In [7]:
# X product-market characteristics
X = np.random.normal(scale = sd_x, size = (J+1,K-1))
X = pd.DataFrame(X, columns = ['x_2', 'x_3'])
X['x_1'] = 1
X['j'] = X.index
X.loc[X['j'] == 0, ['x_1', 'x_2', 'x_3']] = 0

print('shape of X: ', X.shape)
X.head()

shape of X:  (11, 4)


,x_2,x_3,x_1,j
0,0.000000,0.000000,0,0
1,3.158113,0.468290,1,1
2,0.507840,3.493566,1,2
3,1.896881,-0.235369,1,3
4,-0.651372,-0.613362,1,4


## 1.3 M
M is the data frame such that a row contains the product-market specific fixed effect $\xi_{jt}$, marginal cost $c_{jt}$, and price $p_{jt}$. **For now, set $p_{jt}=0$ and fill the equilibrium price later.** In order to change the number of available products in each market, for each market, first draw $J_t$ from a discrete uniform distribution between 1 and $J$. The variation in the available products is important for the identification of the distribution of consumer-level unobserved heterogeneity. Add the row of the outside option to each market whose index is 0 and all the variables take value zero.

- We draw $\xi_{jt}$ from i.i.d. normal distribution with mean 0 and standard deviation $\sigma_{\xi}$: $$\xi_{jt}\sim Normal(0, \sigma_{\xi})$$
  - $\xi_{jt}$ is product-market specific fixed effect. $p_{jt}$ can be correlated with $\xi_{jt}$ but $x_{jt}$s are independent of $\xi_{jt}$.

- The marginal cost of product $j$ in market $t$: $$c_{jt}\sim logNormal(0, \sigma_c),$$ where $\sigma_c$ is referred to as sd_c in the code.


In [8]:
# j = 1, 2, ..., J; t = 1, 2, ..., T
M = pd.DataFrame([(j+1,1) for j in range(J)], columns = ['j', 'temp'])\
.merge(pd.DataFrame([(t+1,1) for t in range(T)], columns = ['t', 'temp']), how = 'outer')
M.drop(columns = 'temp', inplace =True)

M['xi'] = np.random.normal(scale = sd_xi, size = (len(M),1))
M['c'] = np.random.lognormal(sigma = sd_c, size = (len(M),1))
M['p'] = 0

M.sort_values(by = ['t', 'j'], inplace = True)

# for each market, draw 𝐽𝑡 from a discrete uniform distribution between 1 and  𝐽
temp2 = pd.DataFrame()
for t in M.t.unique():
    J_t = np.random.randint(1, J)
    temp1 = M[M.t == t].sample(n = J_t, random_state = 230)
    temp2 = pd.concat([temp1, temp2])
    
# add the row of the outside option to each market whose index is 0 and all the variables take value zero.
# M.loc[M.j == 0, ['xi','c', 'p']] = 0
temp3 = M[['t']].drop_duplicates()
for col in ['j', 'xi', 'c', 'p']:
    temp3[col] = np.zeros(shape = (len(temp3), 1))

M = pd.concat([temp2, temp3])
print('shape of M: ', M.shape)
M.head()

shape of M:  (109, 5)


,j,t,xi,c,p
119,6.0,20,-0.834950,1.050409,0.0
99,5.0,20,0.004312,0.990717,0.0
118,6.0,19,0.558018,0.953264,0.0
98,5.0,19,0.392971,0.957834,0.0
158,8.0,19,0.062806,1.072073,0.0


## 1.4 V
Generate the consumer-level heterogeneity. V is the data frame such that a row contains the vector of shocks to consumer-level heterogeneity, $(v_i', v_i)$ . They are all i.i.d. standard normal random variables.

- $\beta_{itk} = \beta_{0k} + \sigma_k v_{itk},$ where $v_{itk}$ for $k=1,2,\cdots, K$ are i.i.d. standard normal random variables.
  
- $\alpha_{it} = -exp(\mu + \omega v_{it}) = -exp(\mu + \omega^2/2) + [-exp(\mu + \omega v_{it}) + exp(\mu + \omega^2/2)] = \alpha_0 + \tilde{\alpha}_{it},$ where $v_{it}$ are i.i.d. standard normal random variables.

In [8]:
V = pd.DataFrame([(i+1,1) for i in range(N)], columns = ['i', 'temp'])\
.merge(pd.DataFrame([(t+1,1) for t in range(T)], columns = ['t', 'temp']), how = 'outer')
V.drop(columns = 'temp', inplace =True)
for col in ['v_x_1', 'v_x_2', 'v_x_3', 'v_p']:
    V[col] = np.random.normal(size = (N*T, 1))

print('shape of V: ', V.shape)
V.head()

shape of V:  (2000, 6)


,i,t,v_x_1,v_x_2,v_x_3,v_p
0,1,1,0.222337,-0.505634,0.455673,1.069062
1,1,2,0.488172,-1.162879,-1.162611,0.052533
2,1,3,0.568830,1.252223,-0.197146,1.563581
3,1,4,0.781116,-0.655212,0.743714,0.805346
4,1,5,0.933814,0.819826,-1.413157,-0.051522


## 1.5 df
Join $X$, $M$, $V$ and name it df. df is the data frame such that a row contains variables for a consumer about a product that is available in a market.

In [9]:
df = M.merge(X, how = 'left')
df = df.merge(V, how = 'left')
df.sort_values(by = ['t', 'i', 'j'], inplace =True)

print('shape of df: ', df.shape)
df.head()

shape of df:  (12600, 13)


,j,t,xi,c,p,x_2,x_3,x_1,i,v_x_1,v_x_2,v_x_3,v_p
10600,0.0,1,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,0.222337,-0.505634,0.455673,1.069062
10500,5.0,1,0.140941,1.001834,2.141619,4.584802,3.153910,1,1,0.222337,-0.505634,0.455673,1.069062
10400,6.0,1,-0.090019,1.018559,1.896693,4.112289,1.729779,1,1,0.222337,-0.505634,0.455673,1.069062
10601,0.0,1,0.000000,0.000000,0.000000,0.000000,0.000000,0,2,1.217603,0.344503,0.116888,-0.073015
10501,5.0,1,0.140941,1.001834,2.141619,4.584802,3.153910,1,2,1.217603,0.344503,0.116888,-0.073015


## 1.6 e
Draw a vector of preference shocks e whose length is the same as the number of rows of df.

In [10]:
e = np.random.gumbel(size = (len(df), 1))

print('shape of e: ', e.shape)

e

shape of e:  (12600, 1)


array([[ 0.16418068],
       [-0.81040583],
       [-0.55379447],
       ...,
       [ 1.45825932],
       [-1.30366683],
       [ 3.45069786]])

# compute_derivative_share_smooth

In [11]:
def compute_indirect_utility(df, beta, sigma, mu, omega):
    u = (beta[0]+sigma[0]*df.v_x_1)*df.x_1 + (beta[1]+sigma[1]*df.v_x_2)*df.x_2 + (beta[2]+sigma[2]*df.v_x_3)*df.x_3 - \
    np.exp(mu + omega*df.v_p)*df.p + \
    df.xi
    return u

In [13]:
def compute_choice_smooth(df, beta, sigma, mu, omega):
    df['u'] = compute_indirect_utility(df, beta, sigma, mu, omega)
    df['exp_u'] = np.exp(df['u'])
    
    temp = df.groupby(['t']).agg({'exp_u':'sum'}).reset_index()
    temp.columns = ['t', 'sum_exp_u']
    df = df.merge(temp, how = 'left')
    df['q'] = df['exp_u']/df['sum_exp_u']
    df.drop(columns = ['exp_u', 'sum_exp_u'], inplace = True)
    return df

In [15]:
def compute_share_smooth(df, e, beta, sigma, mu, omega):
    df_choice_smooth = compute_choice_smooth(df, beta, sigma, mu, omega)
    
    temp1 = df_choice_smooth.groupby(['j', 't', 'x_1', 'x_2', 'x_3',  'xi', 'c', 'p']).agg({'q': 'mean'}).reset_index()
    temp1.columns = ['j', 't', 'x_1', 'x_2', 'x_3',  'xi', 'c', 'p', 's']

    temp2 = temp1.loc[temp1.j == 0, ['t','s']]
    temp2.columns = ['t', 's_0']
    
    temp1 = temp1.merge(temp2, how = 'left')
    temp1['y'] = np.log(temp1['s']) - np.log(temp1['s_0'])
    return temp1[['j', 't', 'x_2', 'x_3', 'x_1', 'xi', 'c', 'p', 's', 'y']]

The choice probability of product $j$ for consumer $i$ in market $t$ is $\sigma_{ijt}(p_t, x_t, \xi_t)$

 

Suppose that we only observe the (smooth) share data:
$$s_{jt}(p_t, x_t,\xi_t) = \frac{1}{N} \sum_{i=1}^{N}\sigma_{ijt}(p_t, x_t,\xi_t)$$

Thus, 
$$\frac{\partial s_{jt}(p_t, x_t,\xi_t)}{\partial p_{kt}}=\frac{1}{N} \sum_{i=1}^{N}\sigma_{ijt}(p_t, x_t,\xi_t)$$

In [ ]:
def compute_derivative_share_smooth():